# Practical Exam: Grocery Store Sales Analysis

This notebook performs a complete end-to-end analysis for a grocery store chain (FoodYum). It covers data loading, cleaning, exploratory data analysis, feature engineering, simple modeling (monthly forecasting with linear regression), and basket pair analysis. The notebook is written to be robust: if a real CSV file is available it will be loaded, otherwise synthetic data will be generated so the analyses still run.

## 1. Imports and setup

In [ ]:
import os
import random
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

sns.set(style='whitegrid')
pd.set_option('display.max_columns', None)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


## 2. Load data (or generate synthetic dataset if not present)

The code tries to load a CSV from a few common paths. If not found, it generates a synthetic dataset with realistic columns.

In [ ]:
def generate_synthetic_data(n_rows=20000, start_date='2023-01-01', end_date='2024-12-31'):
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)
    days = (end - start).days + 1
    dates = [start + timedelta(days=int(x)) for x in np.random.randint(0, days, size=n_rows)]

    store_ids = [f"S{str(i).zfill(3)}" for i in range(1, 11)]
    product_catalog = []
    categories = ['Produce', 'Meat', 'Dairy', 'Bakery', 'Beverages', 'Frozen', 'Household']
    for pid in range(1, 301):
        cat = np.random.choice(categories, p=[0.18, 0.12, 0.15, 0.12, 0.2, 0.12, 0.11])
        name = f"{cat[:3].upper()}_PROD_{pid}"
        price = round(abs(np.random.normal(loc=5 + categories.index(cat) * 1.5, scale=3)), 2) + 0.5
        product_catalog.append((f"P{str(pid).zfill(4)}", name, cat, price))

    product_df = pd.DataFrame(product_catalog, columns=['product_id', 'product_name', 'category', 'base_price'])

    data = []
    for i in range(n_rows):
        pid, pname, cat, base_price = product_catalog[np.random.randint(0, len(product_catalog))]
        qty = int(np.random.choice([1,1,1,2,2,3,4]))
        price = round(base_price * (1 + np.random.normal(0, 0.05)), 2)
        total = round(price * qty, 2)
        s = np.random.choice(store_ids)
        payment = np.random.choice(['cash', 'credit_card', 'debit_card', 'mobile_payment'], p=[0.15,0.5,0.25,0.1])
        cust_id = f"C{np.random.randint(1,2000):05d}"
        data.append((f'TX{str(i+1).zfill(7)}', dates[i], s, pid, pname, cat, qty, price, total, payment, cust_id))

    df = pd.DataFrame(data, columns=['transaction_id', 'date', 'store_id', 'product_id', 'product_name', 'category', 'quantity', 'price', 'total_sale', 'payment_type', 'customer_id'])
    # add store location mapping
    store_loc = {f"S{str(i).zfill(3)}": np.random.choice(['North','South','East','West']) for i in range(1,11)}
    df['store_region'] = df['store_id'].map(store_loc)
    df['date'] = pd.to_datetime(df['date'])
    return df

possible_paths = [
    'grocery_sales.csv',
    'data/grocery_sales.csv',
    'sales.csv',
    'data/sales.csv'
]
df = None
for p in possible_paths:
    if os.path.exists(p):
        try:
            df = pd.read_csv(p, parse_dates=['date'])
            print(f'Loaded data from {p}')
            break
        except Exception as e:
            print(f'Failed to read {p}: {e}')

if df is None:
    print('No data file found. Generating synthetic dataset...')
    df = generate_synthetic_data(n_rows=20000)

df.head()


## 3. Quick data checks and cleaning

In [ ]:
# Basic info
print('Rows, Columns:', df.shape)
print('\nMissing values per column:')
print(df.isnull().sum())

# Ensure types
df['date'] = pd.to_datetime(df['date'])
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').fillna(1).astype(int)
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0.0)
df['total_sale'] = pd.to_numeric(df['total_sale'], errors='coerce').fillna(df['price'] * df['quantity'])

# Drop rows with essential missing values
essential = ['transaction_id','date','store_id','product_id']
df = df.dropna(subset=essential)

df = df.reset_index(drop=True)
df.sample(5)


## 4. Feature engineering

Add month, day, weekday and revenue per transaction if needed.

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.day_name()
df['month_start'] = df['date'].dt.to_period('M').dt.to_timestamp()
df['revenue'] = df['total_sale']

df[['date','transaction_id','store_id','product_id','category','quantity','price','total_sale']].head()


## 5. Exploratory Data Analysis (EDA)

We'll look at top categories, monthly revenue trend, store performance, and weekday patterns.

In [ ]:
# Top categories by revenue
cat_rev = df.groupby('category')['revenue'].sum().sort_values(ascending=False).reset_index()
cat_rev['pct'] = 100 * cat_rev['revenue'] / cat_rev['revenue'].sum()
cat_rev


In [ ]:
# Plot category revenue
plt.figure(figsize=(8,5))
sns.barplot(data=cat_rev, x='revenue', y='category', palette='viridis')
plt.title('Revenue by Category')
plt.xlabel('Revenue')
plt.tight_layout()
plt.show()


In [ ]:
# Monthly revenue trend
monthly = df.groupby('month_start')['revenue'].sum().reset_index()
monthly = monthly.sort_values('month_start')
plt.figure(figsize=(10,4))
sns.lineplot(data=monthly, x='month_start', y='revenue', marker='o')
plt.title('Monthly Revenue Trend')
plt.ylabel('Revenue')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Store performance
store_perf = df.groupby('store_id')['revenue'].sum().sort_values(ascending=False).reset_index()
store_perf.head(10)


In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(data=store_perf, x='store_id', y='revenue', palette='magma')
plt.title('Revenue by Store')
plt.xlabel('Store')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 6. Top products and baskets

Identify the top selling products and common product pairs (simple co-occurrence analysis at transaction level).

In [ ]:
# Top products by revenue and quantity
top_prod_rev = df.groupby(['product_id','product_name'])['revenue'].sum().sort_values(ascending=False).reset_index().head(20)
top_prod_qty = df.groupby(['product_id','product_name'])['quantity'].sum().sort_values(ascending=False).reset_index().head(20)
top_prod_rev.head()


In [ ]:
# Simple frequent pairs: for each transaction consider all products in that transaction and count pairs
from itertools import combinations

tx_products = df.groupby('transaction_id')['product_id'].apply(lambda x: list(set(x))).reset_index()

pair_counts = {}
for products in tx_products['product_id']:
    if len(products) < 2:
        continue
    for a, b in combinations(sorted(products), 2):
        pair_counts[(a,b)] = pair_counts.get((a,b), 0) + 1

pair_df = pd.DataFrame([{'product_a': a, 'product_b': b, 'count': c} for (a,b),c in pair_counts.items()])
pair_df = pair_df.sort_values('count', ascending=False).head(20)
pair_df.head()


## 7. Simple forecasting: predict next month's revenue using linear regression

This is a toy example that uses the monthly aggregated revenue and fits a linear model on time index to demonstrate basic forecasting.

In [ ]:
monthly = df.groupby('month_start')['revenue'].sum().reset_index().sort_values('month_start')
monthly['month_index'] = (monthly['month_start'] - monthly['month_start'].min()).dt.days // 30

X = monthly[['month_index']].values
y = monthly['revenue'].values

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

print('Train R2:', r2_score(y, y_pred))
print('Train RMSE:', np.sqrt(mean_squared_error(y, y_pred)))

plt.figure(figsize=(10,4))
plt.plot(monthly['month_start'], y, label='Actual', marker='o')
plt.plot(monthly['month_start'], y_pred, label='Linear forecast', linestyle='--')
plt.legend()
plt.title('Monthly revenue and linear fit')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Forecast next 3 months
last_index = monthly['month_index'].max()
future_idx = np.array([[last_index + i] for i in range(1,4)])
future_pred = model.predict(future_idx)
future_dates = [monthly['month_start'].max() + pd.DateOffset(months=i) for i in range(1,4)]
pd.DataFrame({'month_start': future_dates, 'predicted_revenue': future_pred})


## 8. Recommendations and next steps

1. Investigate promotions / holidays: add a promotions and holiday flag to the dataset to capture spikes.
2. Use a time-series model (Prophet, SARIMA) for accurate forecasting and to capture seasonality.
3. Use association-rule mining (apriori) for richer basket insights and build targeted cross-sell campaigns.
4. Build per-store dashboards to monitor KPIs (revenue, avg basket, conversion) and set alerts for anomalies.

This notebook is intentionally self-contained: it will generate data if none is found so you can run and inspect the analysis immediately.

## Appendix: Save cleaned data (optional)
Uncomment and provide a path if you'd like to persist the cleaned and enriched dataset.

In [ ]:
# Uncomment to save
# df.to_csv('cleaned_grocery_sales.csv', index=False)
# print('Saved cleaned_grocery_sales.csv')
print('Notebook finished.')
